In [ ]:
"""
GEE Python API script for Automated Detection of Volcanic Hotspots from Sentinel-2 Archives.

Author: Joyson Estibeiro
Date: 2025
Description:
This script detects volcanic activity in Sentinel-2 L1C imagery archives for an Area of Interest 
using the Normalized Hotspot Index (NHI), based on the method proposed by Marchese et al. (2019). 
Users only need to define a few input parameters in Step 2 (location, date range, cloud cover percentage,
and NHI threshold), after which the workflow is automated. The script filters, processes and visualizes 
images showing thermal anomalies in your area of interest using the Google Earth Engine (GEE) Python API.

# Dependencies:
- earthengine-api (Note: requires Google Earth Engine account + authentication)
- geemap
- geopandas

##  Guide for Setting Custom Input Parameters in STEP 2
1. Location –  Use AOI coordinates OR use a AOI vector file
2. Date Range & Cloud Filter – Set your preferred dates and cloud cover percentage
3. NHI Threshold – Adjust the hotspot filter – The NHI index ranges form -1 to 1 and highlights 
potential hotspots (values > 0). To reduce false positives, use a higher threshold (example: 0.2)
"""

In [2]:
# %pip install earthengine-api geemap geopandas

# ===============================
# STEP 1: Import Required Libraries
# ===============================
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

import geemap
import geopandas as gpd
import math

In [ ]:
# ===============================
# STEP 2: Define input parameters (Area of interest, Date Range, Cloud Cover percentage, NHI Threshold)
# ===============================

aoi_polygon_coords = [[
    (-22.335892, 63.913228),
    (-22.466011, 63.913228),
    (-22.466011, 63.841879),
    (-22.335892, 63.841879),
    (-22.335892, 63.913228)
]]  # Replace by your coordinates in EPSG:4326

aoi_geometry = ee.Geometry.Polygon(aoi_polygon_coords)

# uncomment to use a AOI vector file instead of coordinates
"""
aoi_polygon_file = r"[path_to_your_file]"
gdf = gpd.read_file(aoi_polygon_file)
polygon = gdf.geometry.iloc[0]
aoi_geometry = ee.Geometry.Polygon(polygon)
"""

start_date= '2024-02-01'  # Set your preferred dates
end_date= '2024-12-31'
cloud_cover_percentage = 100  # Set your preferred cloud cover percentage
nhi_threshold= 0.2  # Set your preferred NHI index. Ranges form -1 to 1 and highlights potential hotspots (values > 0).

In [ ]:
# ===============================
# STEP 3: Load Sentinel-2 Data (Product level: L1C)
# ===============================
m = geemap.Map()


original_dataset = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(aoi_geometry) \
    .filterDate(start_date, end_date) \
    .sort('system:time_start') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover_percentage))

print("Number of Images in original_dataset:", original_dataset.size().getInfo())

In [12]:
# ===============================
# STEP 4: Clip images to AOI
# ===============================
def clip_to_aoi(image):
    return image.clip(aoi_geometry)

dataset_1 = original_dataset.map(clip_to_aoi)

# ===============================
# STEP 5: Extracting Acquisition Date in the format: dd-month-YYYY
# ===============================
def add_date(image):
    raw_date = ee.String(image.get('system:index')).slice(0, 8)
    year = raw_date.slice(0, 4)
    month_num = raw_date.slice(4, 6)
    day = raw_date.slice(6, 8)

    months = ee.Dictionary({
        '01': 'January', '02': 'February', '03': 'March', '04': 'April',
        '05': 'May', '06': 'June', '07': 'July', '08': 'August',
        '09': 'September', '10': 'October', '11': 'November', '12': 'December'
    })

    month_name = months.get(month_num)
    formatted_date = day.cat(' ').cat(month_name).cat(' ').cat(year)

    return image.set({'date': formatted_date})

dataset_2 = dataset_1.map(add_date)

# ===============================
# STEP 6: Convert Reflectance to Radiance + Add NHI Bands
# ===============================
def add_nhi_radiance(image):
    cos_theta = ee.Number(image.get('MEAN_SOLAR_ZENITH_ANGLE')).multiply(math.pi / 180).cos()
    R_B12 = ee.Number(image.get('SOLAR_IRRADIANCE_B12'))
    R_B11 = ee.Number(image.get('SOLAR_IRRADIANCE_B11'))
    R_B8A = ee.Number(image.get('SOLAR_IRRADIANCE_B8A'))

    B12 = image.select('B12').divide(10000)
    B11 = image.select('B11').divide(10000)
    B8A = image.select('B8A').divide(10000)

    B12_RD = B12.multiply(R_B12).multiply(cos_theta).divide(math.pi).rename('B12_RD')
    B11_RD = B11.multiply(R_B11).multiply(cos_theta).divide(math.pi).rename('B11_RD')
    B8A_RD = B8A.multiply(R_B8A).multiply(cos_theta).divide(math.pi).rename('B8A_RD')

    NHI_SWIR = B12_RD.subtract(B11_RD).divide(B12_RD.add(B11_RD)).rename('NHI_SWIR_RAD')
    NHI_SWNIR = B11_RD.subtract(B8A_RD).divide(B11_RD.add(B8A_RD)).rename('NHI_SWNIR_RAD')

    return image.addBands([B12_RD, B11_RD, B8A_RD, NHI_SWIR, NHI_SWNIR])

dataset_3 = dataset_2.map(add_nhi_radiance)

# ===============================
# STEP 7: Add Max B12 Radiance Property
# ===============================
def max_B12_RD(image):
    stats = image.select('B12_RD').reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=aoi_geometry,
        scale=20,
        bestEffort=True
    )
    return image.set({'Max_B12_RD': stats.get('B12_RD')})

dataset_4 = dataset_3.map(max_B12_RD)

# ===============================
# STEP 8: Filter by B12_RD > 2
# ===============================
dataset_5 = dataset_4.filter(ee.Filter.gt('Max_B12_RD', 2))

# ===============================
# STEP 9: Add Maximum NHI Index Values
# ===============================
def add_max_indices(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=aoi_geometry,
        scale=20,
        bestEffort=True
    )
    return image.set({
        'max_NHI_SWIR': stats.get('NHI_SWIR_RAD'),
        'max_NHI_SWNIR': stats.get('NHI_SWNIR_RAD')
    })

dataset_6 = dataset_5.map(add_max_indices)

# ===============================
# STEP 10: Final Filter to keep images with prefered NHI threshold
# ===============================
dataset_final = dataset_6.filter(
    ee.Filter.Or(
        ee.Filter.gt('max_NHI_SWIR', nhi_threshold),
        ee.Filter.gt('max_NHI_SWNIR', nhi_threshold)
    )
)

print("Number of images in Final Dataset:", dataset_final.size().getInfo())

# ===============================
# STEP 11: Set Map controls
# ===============================
m.centerObject(aoi_geometry)
m.addLayerControl(position='topright')
# m.add_basemap('SATELLITE')

# ===============================
# STEP 12: Map & Visualize Final Results
# ===============================
dataset_final_range = dataset_final.size().getInfo()
dataset_final_list = dataset_final.toList(dataset_final_range)

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B12', 'B11', 'B8A']
}

for i in range(dataset_final_range):
    image = ee.Image(dataset_final_list.get(i))
    date = image.get('date').getInfo()
    m.addLayer(image, vis_params, f'{date}')

# display MAP
m

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE', 'B12_RD', 'B11_RD', 'B8A_RD', 'NHI_SWIR_RAD', 'NHI_SWNIR_RAD']
Number of images in dataset_final: 26


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…